# fuzzyDist.py
## edge list creation script
## author: J Curry
## date: 09/03/2022

In [1]:
import csv
#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process
import pandas as pd
from tqdm.notebook import tqdm
from alive_progress import alive_bar
import time

from thefuzz import fuzz
from thefuzz import process

import numpy as np

## might not need these??
import random
import gc 
import timeit
import sys
from datetime import datetime as dt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
import re

from scipy.sparse import csr_matrix
#%pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

import time


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fishc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
refsDf = pd.read_csv('citationNetworks/Data/SplitRefs.csv')
#refsDf['ref_title'] = str(refsDf['ref_title'])
refsDf['ref_title'] = refsDf['ref_title'].astype(str)

In [3]:
STOPWORDS = stopwords.words('english')
STOPWORDS = set(STOPWORDS)
    
def text_prepare(text, STOPWORDS):
    """
        text: a string
        
        return: a clean string
    """
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()

    # delete stopwords from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS]) 
    text = text.strip()
    return text

In [4]:
text_prepare(" 'Which camera trap type and how many do I need?'' A review of camera features and study designs for a range of wildlife research applications (2013) Hystrix, 24, pp. 148-156", STOPWORDS)

'camera trap type many need? review camera features study designs range wildlife research applications 2013 hystrix 24 pp. 148-156'

In [5]:
print(type(refsDf['ref_title']))

<class 'pandas.core.series.Series'>


In [6]:
refsDf['ProcessedRef'] = refsDf['ref_title'].apply(lambda x: text_prepare(x, STOPWORDS)) # not working

In [7]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
tf_idf_matrix = tfidf_vectorizer.fit_transform(refsDf['ProcessedRef'])

In [8]:
print(tf_idf_matrix[0])

  (0, 74915)	0.19817021902739246
  (0, 121137)	0.22633767489493806
  (0, 13794)	0.2657340095081818
  (0, 34994)	0.2976221757442021
  (0, 118448)	0.30921328958415467
  (0, 139623)	0.3280220568989637
  (0, 184753)	0.32516350339682326
  (0, 58897)	0.32382239307955824
  (0, 82572)	0.1793236015622716
  (0, 74898)	0.15777839564422713
  (0, 121025)	0.14485437836170506
  (0, 13526)	0.13331285966112455
  (0, 34962)	0.15160325899150712
  (0, 118445)	0.24516488320976343
  (0, 139609)	0.2034815705889608
  (0, 184647)	0.18174581596572859
  (0, 58890)	0.2883937906121029


In [9]:
refsDf['ProcessedRef'].iloc[0]

'computer-assisted system photographic mark-recapture analysis 2012 methods ecol. evol'

In [10]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [11]:
t1 = time.time()

# adjust lower bound: 0.8
# keep top 10 similar results
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)

t = time.time()-t1
print("finished in:", t)

In [2]:
refsDf = pd.read_csv('citationNetworks/Data/SplitRefs.csv')
#print(refsDf.head())
#fuzz.ratio(refsDf["Title"], refsDf["Reference"])



In [3]:
# reduce to unique 
# uniqueRefs = refsDf.drop_duplicates(subset=['ref_title'])

In [7]:
# for i in range(len(uniqueRefs)):
#     title = uniqueRefs.iloc[i,6] # 6 or title
#     refTitle = uniqueRefs.iloc[i,7] # 7 or ref title
#     ID = uniqueRefs.iloc[i,4] # 4 or ID
#     ratio = fuzz.ratio(title, refTitle)
#     if ratio > 50:
#         #append to a df with title, re_title, ID, ratio
#         tmpDf = pd.DataFrame([[ID, title, refTitle, ratio]], columns=list('ID', 'Title', 'RefTitle', 'PercentMatch'))




In [8]:
# for index, row in uniqueRefs.iterrows():
#     title = row['Title'] # 6 or title
#     refTitle = row['ref_title'] # 7 or ref title
#     ID = row['ID'] # 4 or ID
#     ratio = fuzz.ratio(title, refTitle)
#     # print(ratio)
#     # if ratio > 50:
#     #     #append to a df with title, re_title, ID, ratio
#     #     tmpDf = pd.DataFrame([[ID, title, refTitle, ratio]], columns=list('ID', 'Title', 'RefTitle', 'PercentMatch'))
    

In [4]:
nan_value = float("NaN")
uniqueRefs = uniqueRefs.replace("", nan_value)
uniqueRefs = uniqueRefs.dropna(subset=["ref_title"])

In [27]:
# !jupyter nbextension enable --py widgetsnbextension

In [28]:
# miniUniqueRefs = uniqueRefs.iloc[0:10]
# titleDf = uniqueRefs = refsDf.drop_duplicates(subset=['Title'])#
# #tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
# tmpDf = pd.DataFrame(columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID'))
# columns = list(tmpDf)
# newDF = []

# for index, row in tqdm(miniUniqueRefs.iterrows()):
#     title = row['Title'] # 6 or title
#     refTitle = str(row['ref_title']) # 7 or ref title
#     ID = row['ID'] # 4 or ID
#     output = process.extractOne(refTitle, titleDf['Title'], scorer=fuzz.token_sort_ratio)
#     # print(output)
#     ratio = output[1]
#     bestMatch = output[0]
#     bestMatchID = output[2]
#     values = [ID, title, refTitle, ratio, bestMatch, bestMatchID]
#     zipped = zip(columns, values)
#     dictionary = dict(zipped)
#     newDF.append(dictionary)
#     # tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
#     # tmpDf.columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID')
#     # tmpDf.append(tmpDf)
# print(tmpDf)

In [33]:
titleDf = uniqueRefs = refsDf.drop_duplicates(subset=['Title'])#
#tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
tmpDf = pd.DataFrame(columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID'))
columns = list(tmpDf)
newDF = []

#with alive_bar(238148, force_tty=True) as bar:
for index, row in tqdm(uniqueRefs.iterrows()):
    title = row['Title'] # 6 or title
    refTitle = str(row['ref_title']) # 7 or ref title
    ID = row['ID'] # 4 or ID
    output = process.extractOne(refTitle, titleDf['Title'], scorer=fuzz.token_sort_ratio)
    # print(output)
    ratio = output[1]
    bestMatch = output[0]
    bestMatchID = output[2]
    values = [ID, title, refTitle, ratio, bestMatch, bestMatchID]
    zipped = zip(columns, values)
    dictionary = dict(zipped)
    newDF.append(dictionary)
    #tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
    #tmpDf.columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID')
    #tmpDf.append(tmpDf)
#bar()
    

0it [00:00, ?it/s]

In [7]:
miniUniqueRefs = uniqueRefs.iloc[0:10]
df0_names = list(miniUniqueRefs.ref_title.unique())
df1_names = list(uniqueRefs.ref_title.unique())
df2_names = list(uniqueRefs.Title.unique())
print(process.extractOne(str(df0_names), df2_names, scorer=fuzz.token_sort_ratio))
#print(process.extractOne(str(df1_names), df2_names, scorer=fuzz.token_sort_ratio))


("Can community-protected areas conserve biodiversity in human-modified tropical landscapes? The case of terrestrial mammals in Southern Mexico [Aires protégées communautaires peuvent conserver la biodiversité dans les paysages tropicaux modifiés par l'homme? Le cas des mammifères terrestres dans le sud du Mexique.] [New população e extensão do alcance do macaco-aranha marrom-headed equatoriana Criticamente em Perigo (Ateles fusciceps fusciceps) no oeste do Equador] [¿Pueden las áreas protegidas en la comunidad conservar la biodiversidad en paisajes tropicales modificados por el hombre-? El caso de los mamíferos terrestres en el sur de México.]", 38)


In [ ]:
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.ratio(name, x)
        if (score > min_score) & (score > max_score):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [11]:
df1_names = list(uniqueRefs.ref_title.unique())
titlesList = list(uniqueRefs.Title.unique())
refsAsArray = uniqueRefs.to_numpy()
miniUniqueRefs = uniqueRefs.iloc[0:10]
miniArray = miniUniqueRefs.to_numpy()

print(process.extractOne(str(miniArray[:,5]), titlesList, scorer=fuzz.token_sort_ratio))


("Can community-protected areas conserve biodiversity in human-modified tropical landscapes? The case of terrestrial mammals in Southern Mexico [Aires protégées communautaires peuvent conserver la biodiversité dans les paysages tropicaux modifiés par l'homme? Le cas des mammifères terrestres dans le sud du Mexique.] [New população e extensão do alcance do macaco-aranha marrom-headed equatoriana Criticamente em Perigo (Ateles fusciceps fusciceps) no oeste do Equador] [¿Pueden las áreas protegidas en la comunidad conservar la biodiversidad en paisajes tropicales modificados por el hombre-? El caso de los mamíferos terrestres en el sur de México.]", 38)
